In [1]:
import numpy as np #для численных вычислений
import sympy as sp #для символьных вычислений

In [2]:
x0_sym = sp.Matrix([0.23, -0.4])
x0_val = np.array([0.23, -0.4]) #начальная точка в символьном и численном видах
epsilon = 0.001 #погрешность

A = np.array([[6, 0], [0, 3]])
B = np.array([-4, 3])
C = 2 #коэффициенты исходной функции в виде матриц: A - коэфф. при второй степени, B - при первой, C - свободный член

x, y = sp.symbols('x y') #переменные для символьных вычислений
x_ = sp.Matrix([x, y]) #матрица для символьных вычислений
f = 6 * x**2 + 3 * y**2 - 4 * x + 3 * y + 2 #исходная функция
f

6*x**2 - 4*x + 3*y**2 + 3*y + 2

In [3]:
def f(x, A, B, C):
    '''
    Возвращает функцию по заданным матрицам коэффициентов (A, B, C) и двумерной точке (x)
    '''
    return x.dot(A.dot(x)) + B.dot(x) + C

In [4]:
def gradf(x__, A, B, C):
    '''
    Возвращает символьный градиент по заданным матрицам коэффициентов (A, B, C) и символьной двумерной точке (x__)
    '''
    equation, = f(x__, A, B, C)
    return sp.Matrix([sp.diff(equation, v) for v in x_])

def gradf_val(xm_val, x__, A, B, C):
    '''
    Возвращает численный градиент по заданным матрицам коэффициентов (A, B, C), символьной двумерной точке (x__)
    и численной двумерной точке (xm_val)
    '''
    ans = gradf(x__, A, B, C)
    return ans.subs([(x, xm_val[0]), (y, xm_val[1])])

In [5]:
def alpha(xm_sym, xm_val, x__, A, B, C):
    '''
    Возвращает альфа, подходящую для метода наибыстрейшего спуска, по заданным по заданным матрицам коэффициентов (A, B, C),
    символьной двумерной точке (xm_sym) и численной двумерной точке (xm_val)
    '''
    a = sp.Symbol('alpha')
    x_n = gradf_val(xm_val, x__, A, B, C)
    x_n = xm_sym - x_n * a
    q = f(x_n, A, B, C)
    q = sp.simplify(q)
    p = sp.Poly(q[0], a) #преобразуем в в вид полинома, чтобы получить коэффициенты при степенях альфа
    b = p.coeffs()[1]
    a = p.coeffs()[0]
    return (-1) * b / (2 * a) #полученный полином относительно альфа будет иметь вид параболы, ее вершина находится в точке -b/2a

In [6]:
def iter(xm_sym, xm_val, x__, A, B, C):
    '''
    Совершает одну итерацию и возвращает следующую двумерную точку
    x1 = x0 - a * grad(x0)
    '''
    return xm_sym - alpha(xm_sym, xm_val, x__, A, B, C) * gradf_val(xm_val, x__, A, B, C)

In [7]:
def stop(epsilon, xm_val, x__, A, B, C):
    '''
    Проверяет критерий останова
    '''
    return np.max(np.abs(gradf_val(xm_val, x__, A, B, C))) < epsilon

In [8]:
def solve_grad(epsilon, xm_sym, xm_val, x__, A, B, C):
    '''
    Нахождение точки минимума с помощью метода наибыстрейшего градиентного спуска
    '''
    counter = 0
    while not stop(epsilon, xm_val, x_, A, B, C):
        xm_sym = iter(xm_sym, xm_val, x_, A, B, C)
        xm_val = np.array([xm_sym[0], xm_sym[1]])
        counter += 1
    ans_sym = xm_sym
    ans_val = xm_val
    print('Количество итераций: ', counter)
    return ans_sym

solve_grad(epsilon, x0_sym, x0_val, x_, A, B, C)

Количество итераций:  6


Matrix([
[ 0.333295759891293],
[-0.499963638604477]])

In [9]:
def beta(xm_val,xm1_val, x__, A, B, C):
    fm = gradf_val(xm_val, x__, A, B, C)
    fm1 = gradf_val(xm1_val, x__, A, B, C)
    return (np.dot(fm1.T, fm1) / np.dot(fm.T, fm))

In [10]:
def solve_conj(epsilon, xm_sym, xm_val, x__, A, B, C):
    h0 = (-1) * gradf_val(xm_val, x__, A, B, C)
    a = alpha(xm_sym, xm_val, x__, A, B, C)
    xm1_sym = xm_sym + a * h0
    xm1_val = np.array([xm1_sym[0], xm1_sym[1]])
    counter = 0
    while not stop(epsilon, xm_val, x__, A, B, C):
        b = beta(xm_val,xm1_val, x__, A, B, C)
        h1 = (-1) * gradf_val(xm1_val, x__, A, B, C) + b[0][0] * h0
        a = alpha(xm_sym, xm_val, x__, A, B, C)
        xm_sym = xm1_sym
        xm_val = xm1_val
        xm1_sym = xm_sym + a * h1
        xm1_val = np.array([xm1_sym[0], xm1_sym[1]])
        h0 = h1
        counter += 1
    ans_sym = xm_sym
    print('Количество итераций', counter)
    return ans_sym
solve_conj(epsilon, x0_sym, x0_val, x_, A, B, C)

Matrix([[0.230000000000000], [-0.400000000000000]]) [ 0.23 -0.4 ] 0
[0.344162397143301 -0.455239869585468] 1
[0.337553542535960 -0.482556468629144] 2
[0.328931769431526 -0.503531168918986] 3
[0.334265585803250 -0.505588077652600] 4
[0.334578954401802 -0.503093117450620] 5
[0.332555539280186 -0.498362941283364] 6
[0.333155384415534 -0.498330314161812] 7
[0.333720720085567 -0.499445068247741] 8
[0.333227159386182 -0.500467766175164] 9
[0.333264911946591 -0.500372823847726] 10
[0.333413792402844 -0.499973912838674] 11
Количество итераций 11


Matrix([
[ 0.333413792402844],
[-0.499973912838674]])